In [1]:
import torch
import torchaudio
import pyarrow.parquet as pq
from torch import nn
from torch.utils.data import DataLoader, Dataset
from torchvision.transforms import transforms
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torchvision.models as models
import os
from tqdm import tqdm
from torch.nn.functional import one_hot, softmax
import matplotlib.cm as cm
import timm
import random
import albumentations as A
from albumentations.pytorch import ToTensorV2
from PIL import Image

/home/geon/.local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
cmap = cm.get_cmap("viridis")
seed = 42
torch.manual_seed(seed)
random.seed(seed)
np.random.seed(seed)

# If you are using CUDA, add this line as well
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

/tmp/ipykernel_3424735/1323812567.py:1: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cmap = cm.get_cmap("viridis")


In [3]:
from scipy.signal import butter, lfilter

def butter_lowpass_filter(data, cutoff_freq=20, sampling_rate=200, order=4):
    nyquist = 0.5 * sampling_rate
    normal_cutoff = cutoff_freq / nyquist
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    filtered_data = lfilter(b, a, data, axis=0)
    return filtered_data

In [4]:
import torch
import torch.fft as fft
import torch.nn as nn
import torch.nn.functional as F

class DenoisingAutoencoder(nn.Module):
    def __init__(self, input_size):
        super(DenoisingAutoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(10000, 256),
            nn.ReLU(),
            nn.Linear(256, 64),
            nn.ReLU(),
            nn.Linear(64, 32)  # 압축된 표현
        )
        self.decoder = nn.Sequential(
            nn.Linear(32, 64),
            nn.ReLU(),
            nn.Linear(64, 256),
            nn.ReLU(),
            nn.Linear(256, 10000)  # 복원된 출력
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x
    
class FFTReal(nn.Module):
    def forward(self, inputs, dim):
        inputs = inputs.type(torch.complex64)
        fft_result = fft.fft(inputs, dim = dim)
        return torch.real(fft_result)
class SimpleNeuralNetwork(nn.Module):
    def __init__(self, input_size=260, hidden_size1=128, hidden_size2=64, output_size=32):
        super(SimpleNeuralNetwork, self).__init__()
        # 입력층 -> 첫 번째 은닉층
        self.layer1 = nn.Linear(input_size, hidden_size1)
        self.norm1 = nn.LayerNorm(hidden_size1)
        # 첫 번째 은닉층 -> 두 번째 은닉층
        self.layer2 = nn.Linear(hidden_size1, hidden_size2)
        self.norm2 = nn.LayerNorm(hidden_size2)
        # 두 번째 은닉층 -> 출력층
        self.layer3 = nn.Linear(hidden_size2, output_size)

    def forward(self, x):
        # 입력층 -> 첫 번째 은닉층, 활성화 함수 ReLU 적용, 그리고 Layer Normalization
        x = F.relu(self.norm1(self.layer1(x)))
        # 첫 번째 은닉층 -> 두 번째 은닉층, 활성화 함수 ReLU 적용, 그리고 Layer Normalization
        x = F.relu(self.norm2(self.layer2(x)))
        # 두 번째 은닉층 -> 출력층
        x = self.layer3(x)
        return x
class Health_State_Analysis(nn.Module):
    def __init__(self, length_of_signal=10000):
        super(Health_State_Analysis, self).__init__()
        self.length_of_signal = length_of_signal
        self.outlier_threshold = 3


    def hjorth_calculator(self, data):
        batch_size, seq_length, _ = data.shape
        activity = torch.var(data, dim=1)  # Variance (Activity)
        # Calculate first derivative
        diff1 = torch.diff(data, dim=1)
        mobility = torch.sqrt(torch.var(diff1, dim=1) / (1e-10 + torch.var(data, dim=1))) # Mobility
        # Calculate second derivative
        diff2 = torch.diff(diff1, dim=1)
        complexity = torch.sqrt(torch.var(diff2, dim=1) / (1e-10 + (torch.var(diff1, dim=1)) ))  # Complexity
        hjorth_parameters = torch.stack((activity, mobility, complexity), dim=1)
        return activity,mobility,complexity
    
    def compute_skewness(self, data):
        # Calculate mean and standard deviation along the 2560 dimension
        mean = torch.mean(data, dim=1, keepdim=True)
        std = torch.std(data, dim=1, keepdim=True)
        # Compute the third central moment (raw skewness term)
        third_moment = torch.mean(torch.pow((data - mean), 3), dim=1, keepdim=True)
        # Compute the normalized skewness
        skewness = third_moment / torch.pow(std, 3)
        return skewness.squeeze(-1)
    
    def compute_kurtosis(self, data):
        # Calculate mean and standard deviation along the 2560 dimension
        mean = torch.mean(data, dim=1, keepdim=True)
        std = torch.std(data, dim=1, keepdim=True)
        # Compute the fourth central moment (raw kurtosis term)
        fourth_moment = torch.mean(torch.pow((data - mean), 4), dim=1, keepdim=True)
        # Compute the normalized kurtosis
        kurtosis = fourth_moment / torch.pow(std, 4)
        
        return kurtosis.squeeze(-1)
    
    def rainflow_counting(self, data):
      batch_size, seq_length, _ = data.shape
      data_diff = data[:, :, 1:] - data[:, :, :-1]
      up_crossings = torch.where(data_diff[:, :, :-1] > 0)
      down_crossings = torch.where(data_diff[:, :, :-1] < 0)
      count = []
      for i in range(batch_size):
          up_indices = up_crossings[0][up_crossings[1] == i]
          down_indices = down_crossings[0][down_crossings[1] == i]
          if len(up_indices) > 0 and len(down_indices) > 0:
              if up_indices[0] > down_indices[0]:
                  down_indices = down_indices[1:]
              if down_indices[-1] < up_indices[-1]:
                  up_indices = up_indices[:-1]

          cycle_counts = [len(up_indices)]
          count.append(cycle_counts)
      return torch.tensor(count, dtype=torch.float32).to("cuda")

    def zero_crossing_rate(self,signal):
        sign_changes = torch.sum(torch.diff(torch.sign(signal)) != 0)
        zcr = sign_changes.float() / (2 * len(signal))
        return zcr

    def calculate_zcr(self, data):
        batch_size, seq_length, _ = data.shape
        data_flatten = data.view(batch_size, -1)  # 2D로 변환
        zcr_values = (torch.diff(torch.sign(data_flatten)) != 0).sum(dim=1, dtype=torch.float) / (2 * seq_length)
        zcr_values = zcr_values.view(batch_size, -1)  # (batch_size, 1)로 변환
        return zcr_values
    def calculate_mean_crossing_rate(self, data):
        batch_size, seq_length, _ = data.shape
        data_flatten = data.view(batch_size, -1)  # 2D로 변환

        # 평균값 계산
        mean_value = data_flatten.mean(dim=1, keepdim=True)

        # 신호가 평균을 넘는지 여부를 계산
        mean_crossings = torch.diff((data_flatten > mean_value).float(), dim=1)

        # 평균 교차 횟수를 총 시퀀스 길이로 나누어 평균 교차 비율 계산
        mean_crossing_rate = mean_crossings.abs().sum(dim=1) / seq_length

        # 결과 형태를 (batch_size, 1)로 변환
        mean_crossing_rate = mean_crossing_rate.view(batch_size, -1)
        return mean_crossing_rate
    def calculate_velocity(self, data):
        time_diff = 1/self.length_of_signal
        velocity_integral = torch.cumsum(data, dim=1) * time_diff
        # 가속도의 절댓값의 합을 velocity_change 특성으로 사용
        velocity_feature = torch.sum(torch.abs(velocity_integral), dim=1)/self.length_of_signal

        return velocity_feature

    def calculate_displacement(self, data):
        time_diff = 1/self.length_of_signal
        velocity_integral = torch.cumsum(data, dim=1) * time_diff
        displacement_integral = torch.cumsum(velocity_integral, dim=1) * time_diff
        displacement_feature = torch.sum(torch.abs(displacement_integral), dim=1)/self.length_of_signal
        return displacement_feature
        
    def calculate_acceleration(self, data):
        time_diff = 1/self.length_of_signal
        acceleration_feature = torch.sum(torch.abs(data),dim = 1)/self.length_of_signal
        return acceleration_feature

    def calculate_thd(self, signal, fundamental_freq, sampling_rate):
        # FFT를 사용하여 주파수 스펙트럼 계산
        spectrum = torch.fft.fft(signal)

        # 기본 주파수와 관련된 스펙트럼 인덱스 계산
        fundamental_index = (fundamental_freq * signal.shape[-2] / sampling_rate).round().int()
        fundamental_power = torch.abs(spectrum[..., fundamental_index, 0])**2
        # 고조파 스펙트럼의 전력 계산
        fundamental_power = torch.abs(spectrum[..., fundamental_index, 0])**2
        harmonic_power = torch.sum(torch.abs(spectrum[..., fundamental_index * 2::fundamental_index, 0])**2, dim=-1)
        # THD 계산 (백분율로 표시)
        thd = (torch.sqrt(harmonic_power) / (fundamental_power  + 1e-10))*100
        return thd.unsqueeze(-1)

    def calculate_entropy(self, signal):
        batch_size, seq_length, _ = signal.shape

        # Calculate bin edges
        min_val = signal.min(dim=1).values
        max_val = signal.max(dim=1).values
        bin_width = (max_val - min_val) / self.num_bins
        # Calculate histogram indices
        hist_indices = ((signal - min_val.unsqueeze(2)) / bin_width.unsqueeze(2)).long()
        hist_indices = torch.clamp(hist_indices, 0, self.num_bins - 1)
        # Flatten hist_indices for each batch
        hist_indices_flat = hist_indices.view(batch_size, -1)
        # Count occurrences of each bin index
        hist_counts = torch.stack([torch.bincount(hist_idx, minlength=self.num_bins) for hist_idx in hist_indices_flat])
        # Normalize histograms
        histograms = hist_counts / torch.sum(hist_counts, dim=1, keepdim=True)
        # Calculate entropy
        entropy = -torch.sum(histograms * torch.log2(histograms + 1e-10), dim=1)
        return entropy.unsqueeze(-1)

    def forward(self, inputs):
        # 입력 형태: (batch, signal_length, channel_num), 예: (batch, 10000, 20)
        batch_size, signal_length, channel_num = inputs.shape

        # 채널별로 특성을 계산하고 쌓을 리스트
        channel_features = []

        for channel in range(channel_num):
            channel_data = inputs[:, :, channel]

            # 각 채널에 대한 특성 계산
            mean = torch.mean(channel_data, dim=1)
            maximum = torch.max(channel_data, dim=1).values
            minimum = torch.min(channel_data, dim=1).values
            rms = torch.sqrt(torch.mean(channel_data**2, dim=1))
            var = torch.var(channel_data, dim=1)
            skewness = self.compute_skewness(channel_data)  
            kurtosis = self.compute_kurtosis(channel_data)
            shape_factor = rms * self.length_of_signal / torch.sum(torch.abs(channel_data), dim=1)
            crest_factor = torch.max(torch.abs(channel_data), dim=1).values / rms
            impulse_factor = torch.max(torch.abs(channel_data), dim=1).values * self.length_of_signal / torch.sum(torch.abs(channel_data), dim=1)
            outliers = torch.sum((torch.abs(channel_data - channel_data.mean(dim=1, keepdim=True)) > self.outlier_threshold * channel_data.std(dim=1, keepdim=True)).int(), dim=1).unsqueeze(dim=1)
            zcr = self.calculate_zcr(channel_data.unsqueeze(-1))
            mcr = self.calculate_mean_crossing_rate(channel_data.unsqueeze(-1))
            activity, mobility, complexity = self.hjorth_calculator(channel_data.unsqueeze(-1))

            p2p = maximum - minimum

            # 현재 채널에 대한 모든 특성을 쌓기
            channel_feature = torch.stack([mean.unsqueeze(-1), maximum.unsqueeze(-1), minimum.unsqueeze(-1), 
                               p2p.unsqueeze(-1), var.unsqueeze(-1), rms.unsqueeze(-1), 
                               skewness.unsqueeze(-1), kurtosis.unsqueeze(-1), 
                               crest_factor.unsqueeze(-1), shape_factor.unsqueeze(-1), 
                               impulse_factor.unsqueeze(-1),mcr, activity,outliers, mobility, complexity
                             ], dim=1)
            channel_features.append(channel_feature)

        # 모든 채널의 특성을 쌓아서 반환
        stacked_features = torch.stack(channel_features, dim=2).squeeze().view(batch_size,-1)  # (batch, feature_count, channel_num)
        return stacked_features
class CONV_LSTM_Classifier(nn.Module):
    def __init__(
        self,
        in_length: int = 10000,
        output_size: int = 6,
        out_channel1_size = 64,
        out_channel2_size = 256,
        out_channel3_size = 512,
        out_channel4_size = 64,
        kernel1_size = 128,
        kernel2_size = 32,
        kernel3_size = 16,
        kernel4_size = 4,
        lstm_hidden_size : int = 64,
        use_raw_bandpass_filterd = True,
        use_fft_bandpass_filterd = True,
    ):
        super(CONV_LSTM_Classifier, self).__init__()
        
        self.in_channels = 40
        self.in_length = in_length
        self.fft_real = FFTReal()
        self.layer_norm1 = nn.LayerNorm(64)
        self.silu = nn.SiLU()
        self.hs = Health_State_Analysis()
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool1d(kernel_size=2)
        self.dropout  = nn.Dropout(0.1)
        self.use_raw_bandpass_filterd = use_raw_bandpass_filterd
        self.use_fft_bandpass_filterd = use_fft_bandpass_filterd
        # Convolutional layers
        self.conv1 = nn.Conv1d(in_channels=self.in_channels, out_channels=out_channel1_size, kernel_size=kernel1_size, stride=4, padding=1)
        self.batchnorm1 = nn.BatchNorm1d(out_channel1_size)
        self.conv2 = nn.Conv1d(in_channels=out_channel1_size, out_channels=out_channel2_size, kernel_size=kernel2_size, stride=2, padding=1)
        self.batchnorm2 = nn.BatchNorm1d(out_channel2_size)
        self.conv3 = nn.Conv1d(in_channels=out_channel2_size, out_channels=out_channel3_size, kernel_size=kernel3_size, stride=2, padding=1)
        self.batchnorm3 = nn.BatchNorm1d(out_channel3_size)
        self.conv4 = nn.Conv1d(in_channels=out_channel3_size, out_channels=out_channel4_size, kernel_size=kernel4_size, stride=1, padding=1)
        self.batchnorm4 = nn.BatchNorm1d(out_channel4_size)
        # Calculate the output size after convolutions
        self.conv4_out = self.calculate_conv_output_size()
        # LSTM layer
        self.lstm = nn.LSTM(input_size=self.conv4_out, hidden_size=lstm_hidden_size, batch_first=True, bidirectional=True)
        # Since LSTM is bidirectional, we concatenate the outputs, hence the hidden size is doubled
        self.dense1 = nn.Linear(lstm_hidden_size * 2, 64)  # Hidden size is doubled because LSTM is bidirectional
        # Dense layers
        self.dense2 = nn.Linear(64, 64)
        self.dense3 = nn.Linear(64 + 64 + 64 + 64 , 64)
        self.dense4 = nn.Linear(64, 16)
        self.dense5 = nn.Linear(16 , output_size)
        #self.vit = timm.create_model(
            #model_name="vit_base_patch16_224", pretrained=True,
            #num_classes=64, in_chans=1)
        #self.vit512 = timm.create_model('vit_base_patch16_224', img_size=512,num_classes=64, in_chans=1)
        self.eff  = timm.create_model('efficientnet_b0', pretrained=True,num_classes=64, in_chans=1)
        self.resnet = timm.create_model(
            model_name="resnet34d", pretrained=True,
            num_classes=64, in_chans=1)
        #self.dae = DenoisingAutoencoder()
        #self.tabnet = SimpleNeuralNetwork(input_size=640)
        self.vit = timm.create_model('vit_base_patch16_224', pretrained=True,num_classes=64, in_chans=1)

    def calculate_conv_output_size(self):
        # Dummy input for calculating the size of LSTM input
        # This assumes the input size to the first conv layer is (batch_size, channels, sequence_length)
        dummy_input = torch.zeros(1, self.in_channels, self.in_length)
        dummy_output = self.maxpool(self.batchnorm4(self.conv4(self.silu(self.batchnorm3(self.conv3(self.silu(self.maxpool(self.batchnorm2(self.conv2(self.silu(self.batchnorm1(self.conv1(dummy_input)))))))))))))
        return dummy_output.shape[-1]

    def forward(self, eeg, spec_resnet, spec_eff):
        # FFT and Real 


        dynamic_features = eeg

        hs = self.hs(dynamic_features)
        hs = torch.where(torch.isnan(hs), torch.tensor(-10000.0), hs)
        dynamic_features = dynamic_features.transpose(1,2)
        # Apply layers
        z = self.silu((self.dropout(self.conv1(dynamic_features))))

        z = self.silu(self.batchnorm2(self.conv2(z)))
        z = self.maxpool(z)
        z = self.silu(self.batchnorm3(self.dropout(self.conv3(z))))
        z = self.silu(self.batchnorm4(self.conv4(z)))
        z = self.maxpool(z)
        z, _ = self.lstm(z)
        z = z[:, -1, :] 

        z = self.dropout(self.silu(self.dense1(z)))
        #z = self.dense4(self.dropout(self.silu(self.dense3(self.dropout(torch.cat(( self.resnet(spec_resnet), self.vit(spec_eff)), dim=-1))))))
        z = self.dense5(self.dropout(self.silu(self.dense4(self.dropout(self.silu(self.dense3(self.dropout(torch.cat((self.eff(spec_eff),self.vit(spec_eff),self.silu(self.dense2(z)), self.resnet(spec_resnet)), dim=-1)))))))))
        #z = self.dense4(self.dropout(self.silu(self.dense3(self.dropout(torch.cat((self.resnet(spec_resnet), self.vit(spec_eff)), dim=-1))))))

        z = nn.functional.softmax(z, dim = 1)
        return z


    def predict(self, eeg, spec_resnet, spec_eff):
        self.eval()
        with torch.no_grad():
            return self.forward(eeg, spec_resnet, spec_eff)
        


In [5]:
train_df = pd.read_csv('hms-harmful-brain-activity-classification/train.csv')
test_df = pd.read_csv('hms-harmful-brain-activity-classification/test.csv')
train_df.head(10)

,eeg_id,eeg_sub_id,eeg_label_offset_seconds,spectrogram_id,spectrogram_sub_id,spectrogram_label_offset_seconds,label_id,patient_id,expert_consensus,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,other_vote
0,1628180742,0,0.0,353733,0,0.0,127492639,42516,Seizure,3,0,0,0,0,0
1,1628180742,1,6.0,353733,1,6.0,3887563113,42516,Seizure,3,0,0,0,0,0
2,1628180742,2,8.0,353733,2,8.0,1142670488,42516,Seizure,3,0,0,0,0,0
3,1628180742,3,18.0,353733,3,18.0,2718991173,42516,Seizure,3,0,0,0,0,0
4,1628180742,4,24.0,353733,4,24.0,3080632009,42516,Seizure,3,0,0,0,0,0
5,1628180742,5,26.0,353733,5,26.0,2413091605,42516,Seizure,3,0,0,0,0,0
6,1628180742,6,30.0,353733,6,30.0,364593930,42516,Seizure,3,0,0,0,0,0
7,1628180742,7,36.0,353733,7,36.0,3811483573,42516,Seizure,3,0,0,0,0,0
8,1628180742,8,40.0,353733,8,40.0,3388718494,42516,Seizure,3,0,0,0,0,0
9,2277392603,0,0.0,924234,0,0.0,1978807404,30539,GPD,0,0,5,0,1,5


In [6]:
train_df[train_df["eeg_sub_id"] == 0]

,eeg_id,eeg_sub_id,eeg_label_offset_seconds,spectrogram_id,spectrogram_sub_id,spectrogram_label_offset_seconds,label_id,patient_id,expert_consensus,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,other_vote
0,1628180742,0,0.0,353733,0,0.0,127492639,42516,Seizure,3,0,0,0,0,0
9,2277392603,0,0.0,924234,0,0.0,1978807404,30539,GPD,0,0,5,0,1,5
11,722738444,0,0.0,999431,0,0.0,557980729,56885,LRDA,0,1,0,14,0,1
22,387987538,0,0.0,1084844,0,0.0,4099147263,4264,LRDA,0,0,0,3,0,0
28,2175806584,0,0.0,1219001,0,0.0,1963161945,23435,Seizure,3,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106780,3910994355,0,0.0,2146798838,0,0.0,4272062867,28488,LPD,0,9,0,2,0,7
106781,3938393892,0,0.0,2146798838,1,60.0,2587113091,28488,LPD,0,9,0,2,0,7
106783,1850739625,0,0.0,2146798838,3,162.0,2394534310,28488,LPD,0,9,0,2,0,7
106784,1306668185,0,0.0,2147312808,0,0.0,1216355904,57480,LPD,0,3,0,0,0,0


In [7]:
print(len(train_df["eeg_id"].unique()))
print(len(train_df["spectrogram_id"].unique()))

17089
11138


In [8]:
train_df_unique = train_df[train_df["eeg_sub_id"] == 0].reset_index(drop =  True)
train_df_unique

,eeg_id,eeg_sub_id,eeg_label_offset_seconds,spectrogram_id,spectrogram_sub_id,spectrogram_label_offset_seconds,label_id,patient_id,expert_consensus,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,other_vote
0,1628180742,0,0.0,353733,0,0.0,127492639,42516,Seizure,3,0,0,0,0,0
1,2277392603,0,0.0,924234,0,0.0,1978807404,30539,GPD,0,0,5,0,1,5
2,722738444,0,0.0,999431,0,0.0,557980729,56885,LRDA,0,1,0,14,0,1
3,387987538,0,0.0,1084844,0,0.0,4099147263,4264,LRDA,0,0,0,3,0,0
4,2175806584,0,0.0,1219001,0,0.0,1963161945,23435,Seizure,3,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17084,3910994355,0,0.0,2146798838,0,0.0,4272062867,28488,LPD,0,9,0,2,0,7
17085,3938393892,0,0.0,2146798838,1,60.0,2587113091,28488,LPD,0,9,0,2,0,7
17086,1850739625,0,0.0,2146798838,3,162.0,2394534310,28488,LPD,0,9,0,2,0,7
17087,1306668185,0,0.0,2147312808,0,0.0,1216355904,57480,LPD,0,3,0,0,0,0


In [9]:
total_votes = train_df_unique[["seizure_vote","lpd_vote","gpd_vote","lrda_vote","grda_vote","other_vote"]].sum(axis=1)
train_df_unique[["seizure_vote","lpd_vote","gpd_vote","lrda_vote","grda_vote","other_vote"]] = train_df_unique[["seizure_vote","lpd_vote","gpd_vote","lrda_vote","grda_vote","other_vote"]].div(total_votes, axis=0)

In [10]:
train_df_unique.loc[1,["seizure_vote","lpd_vote","gpd_vote","lrda_vote","grda_vote","other_vote"]].values


array([0.0, 0.0, 0.45454545454545453, 0.0, 0.09090909090909091,
       0.45454545454545453], dtype=object)

In [11]:
folder_path = "/home/geon/dev_geon/project/hms-harmful-brain-activity-classification/train_eegs"

# 해당 폴더에서 .parquet 확장자를 가진 파일 목록을 가져옵니다.
parquet_files = [f for f in os.listdir(folder_path) if f.endswith(".parquet")]

# parquet 파일의 개수 출력
print(f"파일 갯수: {len(parquet_files)}")

파일 갯수: 17300


In [12]:
folder_path = "/home/geon/dev_geon/project/hms-harmful-brain-activity-classification/train_spectrograms"

# 해당 폴더에서 .parquet 확장자를 가진 파일 목록을 가져옵니다.
parquet_files = [f for f in os.listdir(folder_path) if f.endswith(".parquet")]

# parquet 파일의 개수 출력
print(f"파일 갯수: {len(parquet_files)}")

파일 갯수: 11138


In [13]:


classes = ['Seizure', 'LPD','GPD',"LRDA","GRDA", "Other"]
mapping = {
    c:i for i, c in enumerate(classes)
}
num_classes = len(classes)

In [14]:
classes

['Seizure', 'LPD', 'GPD', 'LRDA', 'GRDA', 'Other']

In [15]:
import random

In [16]:


transform_resnet = A.Compose([
    A.Resize(p=1.0, height=512, width=512),
    A.HorizontalFlip(p=0.25),  # 25% 확률로 수평 뒤집기 적용
    A.ShiftScaleRotate(shift_limit_x=0.1, shift_limit_y=0, scale_limit=0, rotate_limit=0, p=0.5),
    ToTensorV2(p=1.0)
])
transform_vit = A.Compose([
        A.Resize(p=1.0, height=224, width=224),
        A.HorizontalFlip(p=0.25),  # 25% 확률로 수평 뒤집기 적용
        A.ShiftScaleRotate(shift_limit_x=0.1, shift_limit_y=0, scale_limit=0, rotate_limit=0, p=0.5),
        ToTensorV2(p=1.0)
    ])



In [17]:

class EEG_Dataset(Dataset):
    def __init__(self, data_folder, transform1: A.Compose, transform2: A.Compose,):
        self.data_folder = data_folder
        
        self.transform1 = transform1
        self.transform2 = transform2

    def __len__(self):
        return len(train_df_unique)

    def __getitem__(self, idx):
        idx = idx % len(train_df_unique)  # 인덱스를 데이터프레임 길이 내에 유지합니다.
        
        eeg_path = os.path.join("/home/geon/dev_geon/project/hms-harmful-brain-activity-classification/train_eegs", str(train_df_unique.loc[idx, 'eeg_id']) + '.parquet')
        
        while not len(train_df_unique[train_df_unique['eeg_id'] == int(os.path.basename(eeg_path).split(".")[0])]):
            idx += 1
            idx = idx % len(train_df_unique)
            eeg_path = os.path.join("/home/geon/dev_geon/project/hms-harmful-brain-activity-classification/train_eegs", str(train_df_unique.loc[idx, 'eeg_id']) + '.parquet')
   
        signal_len = len(pd.read_parquet(eeg_path))
        len_minus_default = signal_len - 10000
        i = 0
        if len_minus_default != 0:
            i = random.sample(list(range(len_minus_default)), 1)[0]
        label = mapping[train_df_unique[train_df_unique['eeg_id'] == int(os.path.basename(eeg_path).split(".")[0])]["expert_consensus"].values[0]]
        prob = train_df_unique.loc[idx,["seizure_vote","lpd_vote","gpd_vote","lrda_vote","grda_vote","other_vote"]].values.astype(float)
        eeg = pd.read_parquet(eeg_path).iloc[i:10000 + i, :]
        
        while np.sum(eeg.isna().sum()):
            idx += 1
            idx = idx % len(train_df_unique)
            eeg_path = os.path.join("/home/geon/dev_geon/project/hms-harmful-brain-activity-classification/train_eegs", str(train_df_unique.loc[idx, 'eeg_id']) + '.parquet')
            
            signal_len = len(pd.read_parquet(eeg_path))
            len_minus_default = signal_len - 10000
            i = 0
            if len_minus_default != 0:
                i = random.sample(list(range(len_minus_default)), 1)[0]
            label = mapping[train_df_unique[train_df_unique['eeg_id'] == int(os.path.basename(eeg_path).split(".")[0])]["expert_consensus"].values[0]]
            prob = train_df_unique.loc[idx,["seizure_vote","lpd_vote","gpd_vote","lrda_vote","grda_vote","other_vote"]].values.astype(float)
            eeg = pd.read_parquet(eeg_path).iloc[i:10000 + i, :] # Load parquet file
            
        eeg = np.array(eeg)
        eeg = np.clip(eeg,-1024,1024)/32.0
            
            # BUTTER LOW-PASS FILTER
        eeg_filtered = butter_lowpass_filter(eeg)
        #eeg_fft = np.abs(np.fft.fft(np.array(eeg), axis=0)) / 10000
        eeg = np.concatenate([eeg, eeg_filtered], axis=1)   
        spec_offset = train_df_unique.loc[idx, 'spectrogram_label_offset_seconds']
        
        spec_offset = int(spec_offset // 2)
        if spec_offset!=0:
            spec_offset = np.random.randint(0, spec_offset)
        img_path = os.path.join("/home/geon/dev_geon/project/hms-harmful-brain-activity-classification/train_spectrograms", str(train_df_unique.loc[idx, 'spectrogram_id']) + '.parquet')
        img = pd.read_parquet(img_path)  # shape: (Hz, Time) = (400, 300)
        
        img = np.array(img.fillna(0).values[:, 1:].T.astype("float32")[:, spec_offset: spec_offset + 300])
        img = np.clip(img,np.exp(-4),np.exp(8))
        img = np.log(img)
        eps = 1e-6
        img_mean = img.mean(axis=(0, 1))
        img = img - img_mean
        img_std = img.std(axis=(0, 1))
        img = img / (img_std + eps)

        img = img[..., None] # shape: (Hz, Time) -> (Hz, Time, Channel)
        img_resnet,img_vit = self._apply_transform(img)
        
        return eeg, img_resnet,img_vit, label,prob

    def _apply_transform(self, img: np.ndarray):
        """apply transform to image and mask"""
        transformed1 = self.transform1(image=img)
        transformed2 = self.transform2(image=img)
        img_resnet = transformed1["image"]
        img_vit = transformed2["image"]
        return img_resnet,img_vit


In [18]:
device = 'cuda'

In [19]:
data_folder = "/home/geon/dev_geon/project/hms-harmful-brain-activity-classification/train_eegs"

# Dataset 초기화 시 변환 함수 전달
dataset = EEG_Dataset(data_folder, transform1=transform_resnet,transform2=transform_vit)
train_length = int(0.85 * len(dataset))
val_length = len(dataset) - train_length

print(f'train size: {train_length} \nval size: {val_length}')


    

train_set, val_set = torch.utils.data.random_split(dataset, [train_length, val_length])
train_loader = DataLoader(train_set, batch_size=48, shuffle=True, pin_memory= True)
val_loader = DataLoader(val_set, batch_size=48, pin_memory= True)

train size: 14525 
val size: 2564


In [20]:
conv_lstm = CONV_LSTM_Classifier()
conv_lstm.load_state_dict(torch.load('/home/geon/dev_geon/project/best_model2.pth'))
conv_lstm.to(device)

CONV_LSTM_Classifier(
  (fft_real): FFTReal()
  (layer_norm1): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
  (silu): SiLU()
  (hs): Health_State_Analysis()
  (relu): ReLU()
  (maxpool): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (dropout): Dropout(p=0.1, inplace=False)
  (conv1): Conv1d(40, 64, kernel_size=(128,), stride=(4,), padding=(1,))
  (batchnorm1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv1d(64, 256, kernel_size=(32,), stride=(2,), padding=(1,))
  (batchnorm2): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv1d(256, 512, kernel_size=(16,), stride=(2,), padding=(1,))
  (batchnorm3): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv4): Conv1d(512, 64, kernel_size=(4,), stride=(1,), padding=(1,))
  (batchnorm4): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (lstm):

In [21]:
loss_fn = nn.KLDivLoss(reduction="batchmean")
optimizer = torch.optim.AdamW(conv_lstm.parameters(), lr=0.001)
nll_loss = nn.NLLLoss()


In [22]:
best_kl_divergence = float('inf')
best_accuracy = 0.0  
best_model_state_dict = None
num_epochs = 25

for epoch in range(num_epochs):
    train_loss = 0.0  # 에폭마다 훈련 손실을 추적하기 위해 초기화
    correct = 0       # 정확한 예측의 수를 추적하기 위해 초기화
    total = 0         # 전체 레이블의 수를 추적하기 위해 초기화
    pbar = tqdm(train_loader)
    for eeg, img_resnet, img_vit, labels, prob in pbar:
        labels = labels.to(device)
        labels_onehot = one_hot(labels, num_classes=num_classes).float().to(device)
        prob = prob.float().to(device)
        eeg = eeg.float().to(device)
        img_resnet = img_resnet.float().to(device)
        img_vit = img_vit.float().to(device)
        optimizer.zero_grad()
        outputs = conv_lstm(eeg, img_resnet, img_vit)
        
        loss = loss_fn(outputs.log(), prob)
        nllloss = nll_loss(outputs.log(), labels)

        (loss + nllloss).backward()
        optimizer.step()
        train_loss += loss.item()
        
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        batch_total = labels.size(0)
        batch_correct = (predicted == labels).sum().item()
        
        accuracy = 100 * batch_correct / batch_total
        pbar.set_description(f'Train Loss: {loss.item():.4f} | Accuracy: {accuracy:.2f}%')

    epoch_loss = train_loss / len(train_loader)
    epoch_accuracy = 100 * correct / total
    print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%')


    # Evaluate on validation set
    with torch.no_grad():
        correct = 0
        total = 0
        val_loss = 0
        pbar = tqdm(val_loader)
        for eeg, img_resnet, img_vit, labels, prob in pbar:
            labels = labels.to(device)
            labels_onehot = one_hot(labels, num_classes=num_classes).float()
            prob = prob.float().to(device)
            eeg = eeg.float().to(device)
            img_resnet = img_resnet.float().to(device)
            img_vit = img_vit.float().to(device)
            outputs = conv_lstm(eeg, img_resnet, img_vit)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            loss = loss_fn(outputs.log(), prob).item()
            val_loss += loss
            pbar.set_description(f'val loss {loss}')
            
        accuracy = 100 * correct / total
        kl_divergence = val_loss / len(val_loader)
        if accuracy > best_accuracy:
            # 가장 높은 정확도 업데이트
            best_accuracy = accuracy
            # 현재 가장 좋은 모델 상태 저장
            best_model_state_dict = conv_lstm.state_dict()  # 'conv_lstm'가 모델이라고 가정

            # 모델 상태를 파일로 저장 (예: 'best_model.pth')
            torch.save(conv_lstm.state_dict(), 'best_model2.pth')

        print("Validation Accuracy:", accuracy)
        """
        if kl_divergence < best_kl_divergence:
            # Update the best KL divergence value
            best_kl_divergence = kl_divergence
            # Save the state_dict of the current best model
            best_model_state_dict = conv_lstm.state_dict()  # Assuming 'conv_lstm' is your model
            
            # Save the model to a file (e.g., 'best_model.pth')
            torch.save(conv_lstm.state_dict(), 'best_model2.pth')
        """
        print("Validation KL Divergence:", kl_divergence)

  0%|          | 0/303 [00:00<?, ?it/s]

Train Loss: 0.5545 | Accuracy: 77.08%:  47%|████▋     | 142/303 [09:14<10:03,  3.75s/it]

In [ ]:

class Test_EEG_Dataset(Dataset):
    def __init__(self, data_folder, transform1: A.Compose, transform2: A.Compose,):
        self.data_folder = data_folder
        
        self.transform1 = transform1
        self.transform2 = transform2

    def __len__(self):
        return len(test_df)

    def __getitem__(self, idx):
        idx = idx % len(test_df)  # 인덱스를 데이터프레임 길이 내에 유지합니다.
        
        eeg_path = os.path.join("/home/geon/dev_geon/project/hms-harmful-brain-activity-classification/test_eegs", str(test_df.loc[idx, 'eeg_id']) + '.parquet')
        
        while not len(test_df[test_df['eeg_id'] == int(os.path.basename(eeg_path).split(".")[0])]):
            idx += 1
            idx = idx % len(test_df)
            eeg_path = os.path.join("/home/geon/dev_geon/project/hms-harmful-brain-activity-classification/test_eegs", str(test_df.loc[idx, 'eeg_id']) + '.parquet')
   
        signal_len = len(pd.read_parquet(eeg_path))
        len_minus_default = signal_len - 10000
        i = 0
        if len_minus_default != 0:
            i = random.sample(list(range(len_minus_default)), 1)[0]
        eeg = pd.read_parquet(eeg_path).iloc[i:10000 + i, :].interpolate(method='linear').fillna(method='ffill') # Load parquet file
        
        while np.sum(eeg.isna().sum()):
            idx += 1
            idx = idx % len(test_df)
            eeg_path = os.path.join("/home/geon/dev_geon/project/hms-harmful-brain-activity-classification/test_eegs", str(test_df.loc[idx, 'eeg_id']) + '.parquet')
            
            signal_len = len(pd.read_parquet(eeg_path))
            len_minus_default = signal_len - 10000
            i = 0
            if len_minus_default != 0:
                i = random.sample(list(range(len_minus_default)), 1)[0]
            eeg = pd.read_parquet(eeg_path).iloc[i:10000 + i, :].interpolate(method='linear').fillna(method='ffill') # Load parquet file
            
        eeg = np.array(eeg)
        eeg = np.clip(eeg,-1024,1024)/32.0
        eeg_filtered = butter_lowpass_filter(eeg)
        #eeg_fft = np.abs(np.fft.fft(np.array(eeg), axis=0)) / 10000
        eeg = np.concatenate([eeg, eeg_filtered], axis=1) 
        #eeg_fft = np.abs(np.fft.fft(np.array(eeg), axis=0)) / 10000
        #eeg = np.concatenate([eeg, eeg_fft], axis=1)   
        spec_offset = 0
        
        spec_offset = int(spec_offset // 2)
        
        img_path = os.path.join("/home/geon/dev_geon/project/hms-harmful-brain-activity-classification/test_spectrograms", str(test_df.loc[idx, 'spectrogram_id']) + '.parquet')
        img = pd.read_parquet(img_path)  # shape: (Hz, Time) = (400, 300)
        img = np.array(img.fillna(0).values[:, 1:].T.astype("float32")[:, spec_offset: spec_offset + 300])
        img = np.clip(img,np.exp(-4),np.exp(8))
        img = np.log(img)
        eps = 1e-6
        img_mean = img.mean(axis=(0, 1))
        img = img - img_mean
        img_std = img.std(axis=(0, 1))
        img = img / (img_std + eps)

        img = img[..., None] # shape: (Hz, Time) -> (Hz, Time, Channel)
        img_resnet,img_vit = self._apply_transform(img)
        
        return eeg, img_resnet,img_vit

    def _apply_transform(self, img: np.ndarray):
        """apply transform to image and mask"""
        transformed1 = self.transform1(image=img)
        transformed2 = self.transform2(image=img)
        img_resnet = transformed1["image"]
        img_vit = transformed2["image"]
        return img_resnet,img_vit


In [ ]:
test_dataset = Test_EEG_Dataset(data_folder, transform1=transform_resnet,transform2=transform_vit)
test_loader = DataLoader(test_dataset, batch_size=48, shuffle=False)

In [ ]:
conv_lstm.eval()
out = []
pbar = tqdm(test_loader)
for eeg, img_resnet, img_vit in pbar:
    eeg = eeg.float().to(device)
    img_resnet = img_resnet.float().to(device)
    img_vit = img_vit.float().to(device)
    with torch.no_grad():
        outputs = conv_lstm(eeg, img_resnet, img_vit)
    outputs = outputs.detach().cpu().numpy()
    out.append(outputs)

100%|██████████| 1/1 [00:00<00:00,  3.03it/s]


In [ ]:
conv_lstm.load_state_dict(best_model_state_dict)
conv_lstm.to('cpu')
torch.save(conv_lstm.state_dict(), '/home/geon/dev_geon/project/best_model2.pth')

In [ ]:
conv_lstm.load_state_dict(torch.load('/home/geon/dev_geon/project/best_model2.pth'))
conv_lstm.to(device)

CONV_LSTM_Classifier(
  (fft_real): FFTReal()
  (layer_norm1): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
  (silu): SiLU()
  (hs): Health_State_Analysis()
  (relu): ReLU()
  (maxpool): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (dropout): Dropout(p=0.1, inplace=False)
  (conv1): Conv1d(40, 64, kernel_size=(128,), stride=(4,), padding=(1,))
  (batchnorm1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv1d(64, 256, kernel_size=(32,), stride=(2,), padding=(1,))
  (batchnorm2): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv1d(256, 512, kernel_size=(16,), stride=(2,), padding=(1,))
  (batchnorm3): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv4): Conv1d(512, 64, kernel_size=(4,), stride=(1,), padding=(1,))
  (batchnorm4): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (lstm):

In [ ]:
outputs = np.vstack(out)

In [ ]:
submission = test_df[['eeg_id']].copy()
submission['seizure_vote'] = outputs[:, 0]
submission['lpd_vote'] = outputs[:, 1]
submission['gpd_vote'] = outputs[:, 2]
submission['lrda_vote'] = outputs[:, 3]
submission['grda_vote'] = outputs[:, 4]
submission['other_vote'] = outputs[:, 5]


In [ ]:
submission

,eeg_id,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,other_vote
0,3911565283,0.079535,0.427269,0.006453,0.168518,0.009553,0.308672


In [ ]:
submission.to_csv('submission.csv', index =- False)
submission

,eeg_id,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,other_vote
0,3911565283,0.079535,0.427269,0.006453,0.168518,0.009553,0.308672


In [ ]:
#3911565283	0.220678	0.120944	0.001214	0.22684	0.027411	0.402913